In [28]:
# Install ipywidgets python library
# !pip install ipywidgets

In [29]:
# import libraries
import warnings 
import tensorflow as tf
tf.__version__

import numpy as np
import pandas as pd
import seaborn as sns

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer #word lemma class
from deep_translator import GoogleTranslator # import google translator.
from tqdm import tqdm 
tqdm.pandas()
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import Conv1D,Input, Bidirectional, LSTM, Dense, SpatialDropout1D, Input, Dropout,Embedding,Concatenate, TimeDistributed , BatchNormalization,Flatten,Activation
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report,accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pickle
from tensorflow import keras
lemma = WordNetLemmatizer()
Gtrans= GoogleTranslator(source='auto', target='english')
#initialize and download words,stopword and wordnet word corpus.
words = set(nltk.corpus.words.words())
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')

from ipywidgets import Button, Layout, Box
from ipywidgets import interact, widgets
from IPython.display import display, clear_output

from sklearn.linear_model._base import _preprocess_data
import _pickle as pickle

%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
# Define function for cleaning.
def dataCleaner(text):
    
    #remove x000d from text.
    text=text.replace('_x000D_',' ')
    
    #remove ? from text.
    text=text.replace('?',' ')
    
    #correct acct to account.
    text=text.replace('acct','account')
    
    #correct fw to forward.
    text=text.replace('fw:','forward')
    
    #correct fw to forward.
    text=text.replace(' fw ',' forward ')
    
    #Remove hex chars
    lst = re.findall('==pcap 1 hex s==[\w\W]*==pcap 1 hex e==', text)
    for i in lst:
        text=text.replace(i,' ')
        
    #Remove ascii chars
    lst = re.findall('==pcap 1 ascii s==[\w\W]*==pcap 1 ascii e==', text)
    for i in lst:
        text=text.replace(i,' ')
        
    #Remove correlation data 
    lst = re.findall('\[correlation_data\](.*\n){2}', text)
    for i in lst:
        text=text.replace(i,' ')
       
    #Remove [correlation_data]
    lst = re.findall('\[correlation_data\]', text)
    for i in lst:
        text=text.replace(i,' ')
        
    #Remove [no entry] 
    lst = re.findall('\[no entry\]', text)
    for i in lst:
        text=text.replace(i,' ')
        
    #Remove ascii packet(s):
    text=text.replace('ascii packet(s):',' ')
    
    #Remove hex packet(s)::
    text=text.replace('hex packet(s):',' ')
        
    # Removing white spaces and trimmign the space around words.
    text = re.sub(' +', ' ', text.strip())
    
    #removee tabs
    text=text.replace('\t',' ')
    
    text=text.replace('\r',' ')
    
    #Remove new lines.
    text=text.replace('\n',' ')
    
    # We will use google translate to translate the sentence to english.
    if(len(text)>4000):
        n=3000
        GtransStrings = []
        for index in range(0, len(text), n):
            separated_strings = text[index : index + n]
            GtransStrings.append(Gtrans.translate(separated_strings))
        text=" ".join(GtransStrings)
        
    else:
        if(text!=''):
            text=Gtrans.translate(text)
            
    # Changing to lower case all the words.
    text = text.lower()
    text = text.split()
    
    # Removing english stop words.
    text = [word.strip() for word in text if not word in set(stopwords.words('english'))]
    
    # lemmatizing the words to their base forms
    text = [lemma.lemmatize(word) for word in text]
    
    # Joining back the text.
    text = ' '.join(text)
   
    return text

In [31]:
# Load the models, tokenizer, vectorizer & encoders saved
def loadModelInfo(inp):
    global model1, model2
    global max_len, tokenizer, vectorizer, smallEncoder, FullEncoder
    
    model1 = keras.models.load_model('binaryLSTM_WithGTrans.h5')
    model2 = pickle.load(open('./lr_model.sav', 'rb'))
    max_len = 200
    tokenizer = pickle.load(open('./tokenizer.sav', 'rb'))
    vectorizer = pickle.load(open('./vectorizer.sav', 'rb'))
    smallEncoder = pickle.load(open('./smallEncoder.sav', 'rb'))
    FullEncoder = pickle.load(open('./FullEncoder.sav', 'rb'))
    multiClass_Box.children[1].disabled = False
    print("Model info loaded successfully!")

In [32]:
# Combine the predictions of the binary & the multi classification trained models to predict the assignment group 
# to which the ticket needs to be routed
def getGroupAssignment(text):
    text = dataCleaner(text)
    if text.strip() == '' :
        return 'GRP_0'
    
    X1 = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen = max_len)
    model1Pred = FullEncoder.inverse_transform(np.round(model1.predict(X1),0).astype(int))
    
    if model1Pred == 'GRP_0':
        return 'GRP_0'
    else:
        X2 = vectorizer.transform([text])
        X2 = X2.toarray()
        try:
            model2Pred = smallEncoder.inverse_transform(model2.predict(X2))
            model2Pred = model2Pred[0]
        except:
            model2Pred = 'GRP_0'
        return model2Pred
    
    return 'GRP_0'

In [33]:
# Helper function to collect user input, invoke the getGroupAssignment function, return the response & end the chat session
def botOps():
    try:
        inp = input("Me: Please specify the issue ")
        if inp.lower() == "end":
            return "exit"
        print("Bot: The ticket needs to be routed to this group:", getGroupAssignment(inp))
    
    except EOFError as e:
        print(e)
        return "exit"        
    return ""

In [34]:
# Define the method to invoke the bot operation
# Use the neural network pre-trained model to predict via the bag of words by 
# passing the input string entered by the user along with the words
def startGLBot(self):
    print("Welcome!! I'm the Great Learning Automatic Ticket Assignment Bot. How may I help you today? (Please type 'end' to exit support at any point of time)!!")
    temp = 0
    resp = ''
    while True:                
        resp = botOps()        
        if resp.lower() == "exit":
            break

        try:            
            inp = input("Bot: Was this helpful? Type Yes to exit and No to continue: ")
            if inp.lower() == "yes":
                print("It was a pleasure assisting you. Visit again. Good bye!!")
                break;
            elif inp.lower() == "no":
                botOps() 
            elif inp.lower() == "end":
                print("It was a pleasure assisting you. Visit again. Good bye!!")
                break; 
            else:
                print("Try again. Good bye!!")
                break;
        except EOFError as e:
            print(e)

In [35]:
# Using the ipywidgets, building a simple UI to demonstrate the chatbot functionality

def input_widget(text, callback):
    label = widgets.Label(text)
    
    text = widgets.Text()
    text.on_submit(callback)
    box = widgets.HBox([label, text])
    display(box)
    
def startChat(event=None):
    clear_output(True)
    print("Welcome!! I'm the Great Learning Automatic Ticket Assignment Bot. How may I help you today? \n(Please type 'end' & hit 'enter' to exit support at any point of time)!!")
    input_widget("", process)

def process(event):
    if event.value.lower() == "end" or event.value.lower() == "exit" or event.value.lower() == "yes":
        clear_output(True)
        label = widgets.Label("It was a pleasure assisting you. Visit again by re-executing the cell. Good bye!!")
        display(label)
        return "exit"
    
    print("Bot: The ticket for this issue needs to be routed to the assignment group:", getGroupAssignment(str(event.value)))
    print("Bot: Was this helpful? Type Yes to exit and No to continue:") 
    
    if event.value.lower() == "no":
        event.value = ""
        startChat()
        
    event.value = ""    

In [36]:
# Define the UI layout with only two buttons: Load Model & Chatbot Assistant
items_layout = Layout(width='50%', height='80px') 
box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')

words = ['Load Composite Trained Model', 'Launch GL Auto Ticket Assignment Chatbot Assistant']
items = [Button(description=word, layout=items_layout, button_style='danger', disabled=True) for word in words]
global multiClass_Box
multiClass_Box = Box(children=items, layout=box_layout)

multiClass_Box.children[0].disabled = False
multiClass_Box.children[0].on_click(loadModelInfo)
multiClass_Box.children[1].on_click(startChat)

In [37]:
# Invoke the chat bot operation from the notebook console 
loadModelInfo('')
startGLBot('')

Model info loaded successfully!
Welcome!! I'm the Great Learning Automatic Ticket Assignment Bot. How may I help you today? (Please type 'end' to exit support at any point of time)!!
Me: Please specify the issue exit
Bot: The ticket needs to be routed to this group: GRP_0
Bot: Was this helpful? Type Yes to exit and No to continue: yes
It was a pleasure assisting you. Visit again. Good bye!!


In [38]:
# Log only the errors & suppress warnings from tensorflow to improve readability
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [39]:
# Display the UI for the chatbot
multiClass_Box

Box(children=(Button(button_style='danger', description='Load Composite Trained Model', layout=Layout(height='…

In [40]:
multiClass_Box

Box(children=(Button(button_style='danger', description='Load Composite Trained Model', layout=Layout(height='…